# Library

In [1]:
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import (
    mean_squared_error, 
    mean_absolute_error, 
    r2_score, 
    mean_absolute_percentage_error
)
import numpy as np
warnings.filterwarnings('ignore')

In [8]:
# Memuat Data dan membersihkan kolom yang mengandung '%' serta konversi ke float
data = pd.read_csv('data.csv').replace('%', '', regex=True).astype({
    'Inflation': float, 
    'BIRate_inPercentage': float, 
    'USPrimeRate_inPercentage': float,
    'ExchangeRate_USDtoIDR': float, 
    'ForeignExchangeReserves_inMillionUSD': float, 
    'SuratUtangNegaraSUN_inMillionIDR': float
})

# Konversi 'Date' ke datetime, ekstraksi 'Year' dan 'Month'
data['Date'] = pd.to_datetime(data['Date'], format='%m/%d/%Y')
data['Year'], data['Month'] = data['Date'].dt.year, data['Date'].dt.month

# Isi nilai kosong dengan rata-rata dan pilih variabel dependen
data.fillna(data.mean(), inplace=True)
label = data[['Inflation']]
dependent_variables = ['Year', 
                       'Month', 
                       'BIRate_inPercentage', 
                       'USPrimeRate_inPercentage',
                       'ExchangeRate_USDtoIDR', 
                       'ForeignExchangeReserves_inMillionUSD',
                       'SuratUtangNegaraSUN_inMillionIDR']
data = data[dependent_variables]
# Gabungkan data dan label
final_data = pd.concat([data, label], axis=1)

# Simpan dalam bentuk CSV
final_data.to_csv('final_data.csv', index=False)

In [3]:
data.head()

,Year,Month,BIRate_inPercentage,USPrimeRate_inPercentage,ExchangeRate_USDtoIDR,ForeignExchangeReserves_inMillionUSD,SuratUtangNegaraSUN_inMillionIDR
0,2010,1,6.5,3.25,9365.0,69562.47,577665.0
1,2010,2,6.5,3.25,9335.0,69730.60,577303.0
2,2010,3,6.5,3.25,9115.0,71823.21,571649.0
3,2010,4,6.5,3.25,9012.0,78582.77,580019.0
4,2010,5,6.5,3.25,9180.0,74587.29,585666.0


In [4]:
label.head()

,Inflation
0,3.72
1,3.81
2,3.43
3,3.91
4,4.16


In [5]:
# Split data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(
    data, label, test_size=0.2, random_state=42
)

# Inisialisasi model regresi linear
model = LinearRegression()

# Latih model
model.fit(X_train, y_train)

# Prediksi berdasarkan data testing
y_pred = model.predict(X_test)

# Fungsi untuk menghitung Adjusted R-squared
def adjusted_r2(r2, n, p):
    return 1 - (1 - r2) * (n - 1) / (n - p - 1)

# Evaluasi hasil model
r2 = r2_score(y_test, y_pred)
adj_r2 = adjusted_r2(r2, X_test.shape[0], X_test.shape[1])
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Output hasil evaluasi
print("Koefisien regresi     :", model.coef_)
print("Intercept             :", model.intercept_)
print("Mean Squared Error    :", mean_squared_error(y_test, y_pred))
print("R-squared             :", r2)
print("Adjusted R-squared    :", adj_r2)
print("Mean Absolute Error   :", mean_absolute_error(y_test, y_pred))
print("Root Mean Squared Error:", rmse)
print("Mean Absolute Percentage Error:", mean_absolute_percentage_error(y_test, y_pred))


Koefisien regresi     : [[-5.45625480e-01 -3.47592344e-02  8.96528086e-01 -1.20348052e-01
   2.91602978e-04  2.21674939e-05  8.84325048e-07]]
Intercept             : [1092.10803798]
Mean Squared Error    : 1.6954666394908402
R-squared             : 0.5107911750169216
Adjusted R-squared    : 0.38395925742871606
Mean Absolute Error   : 0.9829279114143233
Root Mean Squared Error: 1.302100856113243
Mean Absolute Percentage Error: 0.24791987548109995


In [6]:
# Menambahkan kolom Status Covid
data['Status Covid'] = 0  

# Mengatur nilai 1 untuk rentang waktu Maret 2020 hingga Maret 2022
mask = ((data['Year'] == 2020) & (data['Month'] >= 3)) | \
       ((data['Year'] == 2021)) | \
       ((data['Year'] == 2022) & (data['Month'] <= 3))
data.loc[mask, 'Status Covid'] = 1
data


,Year,Month,BIRate_inPercentage,USPrimeRate_inPercentage,ExchangeRate_USDtoIDR,ForeignExchangeReserves_inMillionUSD,SuratUtangNegaraSUN_inMillionIDR,Status Covid
0,2010,1,6.50,3.25,9365.0,69562.47,5.776650e+05,0
1,2010,2,6.50,3.25,9335.0,69730.60,5.773030e+05,0
2,2010,3,6.50,3.25,9115.0,71823.21,5.716490e+05,0
3,2010,4,6.50,3.25,9012.0,78582.77,5.800190e+05,0
4,2010,5,6.50,3.25,9180.0,74587.29,5.856660e+05,0
...,...,...,...,...,...,...,...,...
168,2024,1,6.00,8.50,15796.0,145052.00,4.667689e+06,0
169,2024,2,6.00,8.50,15673.0,144040.00,4.726066e+06,0
170,2024,3,6.00,8.50,15853.0,140390.00,4.648698e+06,0
171,2024,4,6.25,8.50,16249.0,136218.00,4.644098e+06,0


In [7]:
# Split data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(
    data, label, test_size=0.2, random_state=42
)

# Inisialisasi model regresi linear
model = LinearRegression()

# Latih model
model.fit(X_train, y_train)

# Prediksi berdasarkan data testing
y_pred = model.predict(X_test)

# Fungsi untuk menghitung Adjusted R-squared
def adjusted_r2(r2, n, p):
    return 1 - (1 - r2) * (n - 1) / (n - p - 1)

# Evaluasi hasil model
r2 = r2_score(y_test, y_pred)
adj_r2 = adjusted_r2(r2, X_test.shape[0], X_test.shape[1])
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Output hasil evaluasi
print("Koefisien regresi     :", model.coef_)
print("Intercept             :", model.intercept_)
print("Mean Squared Error    :", mean_squared_error(y_test, y_pred))
print("R-squared             :", r2)
print("Adjusted R-squared    :", adj_r2)
print("Mean Absolute Error   :", mean_absolute_error(y_test, y_pred))
print("Root Mean Squared Error:", rmse)
print("Mean Absolute Percentage Error:", mean_absolute_percentage_error(y_test, y_pred))


Koefisien regresi     : [[-5.37072807e-01 -3.28500755e-02  8.43920200e-01 -7.69671838e-01
   2.83547312e-04  2.70934864e-05  1.63493967e-06 -2.85987262e+00]]
Intercept             : [1076.29955591]
Mean Squared Error    : 1.5680371705939686
R-squared             : 0.5475595898563768
Adjusted R-squared    : 0.4083471559660311
Mean Absolute Error   : 0.8920025538474771
Root Mean Squared Error: 1.2522129094502934
Mean Absolute Percentage Error: 0.21004997133069528
